In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import pandas as pd
from collections import OrderedDict
from shapely.geometry import Point, LineString
import scalebar

import nept
import scalebar

from loading_data import get_data
from utils_maze import get_xyedges, find_zones, speed_threshold
from analyze_decode import get_edges, point_in_zones

import info.r066d1 as info

In [ ]:
output_filepath = 'E:/code/emi_shortcut/plots/intermediate'
pickle_filepath = 'E:/code/emi_shortcut/cache/pickled'

In [ ]:
import info.r063d2 as info

In [ ]:
events, position, spikes, lfp, lfp_theta = get_data(info)
xedges, yedges = nept.get_xyedges(position)

experiment_time = 'pauseA'
filename = '_decode-' + experiment_time + '.pkl'
decode_filename = info.session_id + filename
pickled_decoded = os.path.join(pickle_filepath, decode_filename)

with open(pickled_decoded, 'rb') as fileobj:
    decode = pickle.load(fileobj)

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)
histogram, xs, ys = np.histogram2d(decode['decoded'].x, decode['decoded'].y, bins=xx.shape)

cmap = plt.cm.get_cmap('pink_r', 11) 
pp = plt.pcolormesh(yy, xx, histogram, cmap=cmap)
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
neurons_filename = info.session_id + '_neurons.pkl'
pickled_neurons = os.path.join(pickle_filepath, neurons_filename)
with open(pickled_neurons, 'rb') as fileobj:
    neurons = pickle.load(fileobj)
    
experiment_time = 'phase3'
speed_limit = 0.4
shuffle_id = False
min_length = 3

In [ ]:
print('decoding:', info.session_id)

track_times = ['phase1', 'phase2', 'phase3', 'tracks']
pedestal_times = ['pauseA', 'pauseB', 'prerecord', 'postrecord']

events, position, spikes, lfp, lfp_theta = get_data(info)
xedges, yedges = nept.get_xyedges(position)

In [ ]:
exp_start = info.task_times[experiment_time].start
exp_stop = info.task_times[experiment_time].stop

decode_spikes = neurons.time_slice(exp_start, exp_stop)

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)

for i, tc in enumerate(neurons.tuning_curves[:5]):
    print(i)
    pp = plt.pcolormesh(xx, yy, tc, cmap='pink_r')
    plt.colorbar(pp)
    plt.axis('off')
    plt.show()

In [ ]:
fields = [3, 14, 27, 30]
for tc_idx in fields:
    filename = info.session_id + '-tuning_curve-' + str(tc_idx) + '.png' 
    pp = plt.pcolormesh(xx, yy, neurons.tuning_curves[tc_idx], cmap='pink_r')
    plt.colorbar(pp)
    plt.axis('off')
    plt.show()
#     plt.savefig(os.path.join(output_filepath, filename), transpatent=True)
#     plt.close()

In [ ]:
multiple_tuning_curves = np.zeros(neurons.tuning_shape)
multiple_tuning_curves += neurons.tuning_curves[3]
multiple_tuning_curves += neurons.tuning_curves[14]
multiple_tuning_curves += neurons.tuning_curves[27]
multiple_tuning_curves += neurons.tuning_curves[30]

pp = plt.pcolormesh(xx, yy, multiple_tuning_curves, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()
# plt.savefig(os.path.join(output_filepath, info.session_id + '-tuning_curve-3.png'))
# plt.savefig(os.path.join(output_filepath, info.session_id + '-tuning_curve-3-14.png'))
# plt.savefig(os.path.join(output_filepath, info.session_id + '-tuning_curve-3-14-27.png'))
# plt.savefig(os.path.join(output_filepath, info.session_id + '-tuning_curve-3-14-27-30.png'))
# plt.close()

In [ ]:
all_tuning_curves = np.zeros(neurons.tuning_shape)
for i in range(neurons.n_neurons):
    all_tuning_curves += neurons.tuning_curves[i]

pp = plt.pcolormesh(xx, yy, all_tuning_curves, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
# plt.show()
plt.savefig(os.path.join(output_filepath, info.session_id + '-tuning_curve-all.png'))
plt.close()

In [ ]:
exp_start = info.task_times[experiment_time].start
exp_stop = info.task_times[experiment_time].stop

spikes = neurons.time_slice(exp_start, exp_stop)

In [ ]:
if experiment_time in track_times:
    run_position = speed_threshold(position, speed_limit=speed_limit)
else:
    run_position = position
    
exp_position = run_position.time_slice(exp_start, exp_stop)

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)
histogram, xs, ys = np.histogram2d(exp_position.x, exp_position.y, 
                                   bins=xx.shape)

cmap = plt.cm.get_cmap('pink_r', 11)
pp = plt.pcolormesh(yy, xx, histogram, cmap=cmap)
plt.colorbar(pp)
plt.axis('off')
plt.show()
# plt.savefig(os.path.join(output_filepath, info.session_id + '-position2d.png'))
# plt.close()

In [ ]:
if shuffle_id:
    random.shuffle(tuning_curve)

In [ ]:
if experiment_time in track_times:
    epochs_interest = nept.Epoch(np.hstack([exp_start, exp_stop]))
elif experiment_time in pedestal_times:
    sliced_lfp = lfp.time_slice(exp_start, exp_stop)

    z_thresh = 3.0
    power_thresh = 5.0
    merge_thresh = 0.02
    min_length = 0.01
    swrs = nept.detect_swr_hilbert(sliced_lfp, fs=info.fs, thresh=(140.0, 250.0), z_thresh=z_thresh,
                                  power_thresh=power_thresh, merge_thresh=merge_thresh, min_length=min_length)

    epochs_interest = nept.find_multi_in_epochs(decode_spikes, swrs, min_involved=4)
    if epochs_interest.n_epochs == 0:
        epochs_interest = nept.find_multi_in_epochs(decode_spikes, swrs, min_involved=1)
else:
    raise ValueError("unrecognized experimental phase. Must be in ['prerecord', 'phase1', 'pauseA', 'phase2', "
                     "'pauseB', phase3', 'postrecord'].")

In [ ]:
counts_binsize = 0.025
time_edges = get_edges(exp_position, counts_binsize, lastbin=True)
counts = nept.get_counts(decode_spikes, time_edges, gaussian_std=0.005)

In [ ]:
# pp = plt.pcolormesh(counts.data, cmap='pink_r')
# plt.colorbar(pp)
# plt.axis('off')
# plt.show()

In [ ]:
tc_shape = neurons.tuning_curves.shape
decoding_tc = neurons.tuning_curves.reshape(tc_shape[0], 
                                            tc_shape[1] * tc_shape[2])

likelihood = nept.bayesian_prob(counts, decoding_tc, counts_binsize)

In [ ]:
xcenters = (xedges[1:] + xedges[:-1]) / 2.
ycenters = (yedges[1:] + yedges[:-1]) / 2.
xy_centers = nept.cartesian(xcenters, ycenters)

time_centers = (time_edges[1:] + time_edges[:-1]) / 2.

decoded = nept.decode_location(likelihood, xy_centers, time_centers)
nan_idx = np.logical_and(np.isnan(decoded.x), np.isnan(decoded.y))
decoded = decoded[~nan_idx]

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)
histogram, xs, ys = np.histogram2d(decoded.x, decoded.y, bins=xx.shape)

pp = plt.pcolormesh(yy, xx, histogram, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()
# plt.savefig(os.path.join(output_filepath, info.session_id + '-decoded2d.png'))
# plt.close()

In [ ]:
if not decoded.isempty:
    sequences = nept.remove_teleports(decoded, speed_thresh=40, min_length=min_length)
    decoded_epochs = sequences.intersect(epochs_interest)
    decoded = decoded[decoded_epochs]
else:
    raise ValueError("decoded cannot be empty.")

In [ ]:
zones = find_zones(info, remove_feeder=True, expand_by=8)
decoded_zones = point_in_zones(decoded, zones)

In [ ]:
keys = ['u', 'shortcut', 'novel']
errors = dict()
actual_position = dict()
if experiment_time in ['phase1', 'phase2', 'phase3', 'tracks']:
    for trajectory in keys:
        actual_x = np.interp(decoded_zones[trajectory].time, exp_position.time, exp_position.x)
        actual_y = np.interp(decoded_zones[trajectory].time, exp_position.time, exp_position.y)
        actual_position[trajectory] = nept.Position(np.hstack((actual_x[..., np.newaxis],
                                                              actual_y[..., np.newaxis])),
                                                   decoded_zones[trajectory].time)

        if actual_position[trajectory].n_samples > 0:
            errors[trajectory] = actual_position[trajectory].distance(decoded_zones[trajectory])
else:
    for trajectory in decoded_zones:
        errors[trajectory] = []
        actual_position[trajectory] = []

output = dict()
output['zones'] = decoded_zones
output['errors'] = errors
output['times'] = len(time_centers)
output['actual'] = actual_position
output['decoded'] = decoded

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)
histogram, xs, ys = np.histogram2d(output['decoded'].x, output['decoded'].y, bins=xx.shape)

pp = plt.pcolormesh(yy, xx, histogram, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
print('u error mean', np.mean(output['errors']['u']))
print('shortcut error mean', np.mean(output['errors']['shortcut']))
print('novel error mean', np.mean(output['errors']['novel']))

In [ ]:
len(xedges)

In [ ]:
r = nept.find_nearest_idx(xedges, 99.)

In [ ]:
r

In [ ]:
run_position = speed_threshold(position, speed_limit=0.4)

In [ ]:
histogram, xs, ys = np.histogram2d(run_position.x, run_position.y, bins=xx.shape)

pp = plt.pcolormesh(yy, xx, histogram, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
histogram[21][2]

In [ ]:

actual_x = np.interp(decoded.time, exp_position.time, exp_position.x)
actual_y = np.interp(decoded.time, exp_position.time, exp_position.y)
actual_pos = nept.Position(np.hstack((actual_x[..., np.newaxis], actual_y[..., np.newaxis])), decoded.time)
err = actual_pos.distance(decoded)


In [ ]:
np.where(histogram > 2500)

In [ ]:
histogram, xs, ys = np.histogram2d(actual_pos.x, actual_pos.y, bins=xx.shape)

pp = plt.pcolormesh(yy, xx, histogram, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
xedges, yedges = nept.get_xyedges(actual_pos)
xx, yy = np.meshgrid(xedges, yedges)

In [ ]:
error_hist_byactual = np.zeros((len(yedges), len(xedges)))
n_hist_byactual = np.ones((len(yedges), len(xedges)))

error_x = []
error_y = []
for error, x, y in zip(err, actual_pos.x, actual_pos.y):
    x_idx = nept.find_nearest_idx(xedges, x)
    y_idx = nept.find_nearest_idx(yedges, y)
    error_hist_byactual[y_idx][x_idx] += error
    n_hist_byactual[y_idx][x_idx] += 1

error_byactual = error_hist_byactual / n_hist_byactual

In [ ]:
pp = plt.pcolormesh(xx, yy, n_hist_byactual, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
pp = plt.pcolormesh(xx, yy, error_hist_byactual, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
pp = plt.pcolormesh(xx, yy, error_byactual, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
# plt.show()

filename = 'error_actual.png'
plt.savefig(os.path.join(output_filepath, filename), transparent=True)
plt.close()

In [ ]:
histogram, xs, ys = np.histogram2d(decoded.x, decoded.y, bins=xx.shape)

pp = plt.pcolormesh(yy, xx, histogram, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
xedges, yedges = nept.get_xyedges(decoded)
xx, yy = np.meshgrid(xedges, yedges)

In [ ]:
error_hist_bydecode = np.zeros((len(yedges), len(xedges)))
n_hist_bydecode = np.ones((len(yedges), len(xedges)))

error_x = []
error_y = []
for error, x, y in zip(err, decoded.x, decoded.y):
    x_idx = nept.find_nearest_idx(xedges, x)
    y_idx = nept.find_nearest_idx(yedges, y)
    error_hist_bydecode[y_idx][x_idx] += error
    n_hist_bydecode[y_idx][x_idx] += 1

error_bydecode = error_hist_bydecode / n_hist_bydecode

In [ ]:
pp = plt.pcolormesh(xx, yy, error_hist_bydecode, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
pp = plt.pcolormesh(xx, yy, n_hist_bydecode, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
pp = plt.pcolormesh(xx, yy, error_bydecode, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

# filename = 'error_decoded.png'
# plt.savefig(os.path.join(output_filepath, filename), transparent=True)
# plt.close()

In [ ]:
events, position, spikes, lfp, lfp_theta = get_data(info)
xedges, yedges = nept.get_xyedges(position)

run_pos = speed_threshold(position, speed_limit=speed_limit)

position_2d, pos_xedges, pos_yedges = np.histogram2d(run_pos.y, run_pos.x, bins=[yedges, xedges])
position_2d *= sampling_rate
shape = position_2d.shape

In [ ]:
fig = plt.figure(figsize=(6, 4))
ax = plt.subplot(111)
pp = plt.pcolormesh(xx, yy, position_2d, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
position_2d[21][2]

In [ ]:
fig = plt.figure(figsize=(6, 4))
ax = plt.subplot(111)
plt.plot(decoded.time, decoded.x, '#9970ab', ms=1)
plt.plot(position.time, position.x, 'k', ms=3)
plt.xlim(26100, 26150)
scalebar.add_scalebar(ax, matchy=False, bbox_transform=fig.transFigure,
                      bbox_to_anchor=(0.9, 0.1), units='ms')
plt.axis('off')
plt.show()
# plt.savefig(os.path.join(output_filepath, info.session_id + '-decoded-x.png'), transparent=True)
# plt.close()

In [ ]:
fig = plt.figure(figsize=(6, 4))
ax = plt.subplot(111)
plt.plot(decoded.time, decoded.y, '#5aae61', ms=1)
plt.plot(position.time, position.y, 'k', ms=3)
plt.xlim(26100, 26150)
scalebar.add_scalebar(ax, matchy=False, bbox_transform=fig.transFigure,
                         bbox_to_anchor=(0.9, 0.1), units='ms')
plt.axis('off')
plt.show()
# plt.savefig(os.path.join(output_filepath, info.session_id + '-decoded-y.png'), transparent=True)
# plt.close()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import pandas as pd
from collections import OrderedDict
from shapely.geometry import Point, LineString

import nept

from loading_data import get_data
from utils_maze import get_xyedges, find_zones, speed_threshold
from analyze_decode import get_edges, point_in_zones

output_filepath = 'E:/code/emi_shortcut/plots/intermediate'
pickle_filepath = 'E:/code/emi_shortcut/cache/pickled'

from utils_fields import categorize_fields
from analyze_tuning_curves import get_odd_firing_idx
from plot_sequence_raster import plot_sequence

In [ ]:
events, position, spikes, lfp, lfp_theta = get_data(info)
xedges, yedges = nept.get_xyedges(position)
zones = find_zones(info, remove_feeder=True, expand_by=8)

neurons_filename = info.session_id + '_neurons.pkl'
pickled_neurons = os.path.join(pickle_filepath, neurons_filename)
with open(pickled_neurons, 'rb') as fileobj:
    neurons = pickle.load(fileobj)
    
xcenters = (xedges[1:] + xedges[:-1]) / 2.
ycenters = (yedges[1:] + yedges[:-1]) / 2.
xy_centers = nept.cartesian(xcenters, ycenters)

In [ ]:
field_thresh = 1.0
fields_tunings = categorize_fields(neurons.tuning_curves, zones, xedges, yedges, 
                                   field_thresh=field_thresh)

u_line = LineString(info.u_trajectory)

u_dist = []
for neuron in fields_tunings['u']:
    yy = ycenters[np.where(fields_tunings['u'][neuron] == fields_tunings['u'][neuron].max())[0][0]]
    xx = xcenters[np.where(fields_tunings['u'][neuron] == fields_tunings['u'][neuron].max())[1][0]]

    pt = Point(xx, yy)
    if zones['u'].contains(pt):
        u_dist.append((u_line.project(pt), neuron))
        
ordered_dist_u = sorted(u_dist, key=lambda x:x[0])
sort_idx = []
for neuron in ordered_dist_u:
    sort_idx.append(neuron[1])
    
sort_spikes = []
sort_tuning_curves = []
for neuron in sort_idx:
    sort_tuning_curves.append(fields_tunings['u'][neuron])
    sort_spikes.append(neurons.spikes[neuron])
    
odd_firing_idx = get_odd_firing_idx(sort_tuning_curves, max_mean_firing=2)

ordered_spikes = []
ordered_fields =[]
for i, neuron in enumerate(sort_spikes):
    if i not in odd_firing_idx:
        ordered_spikes.append(neuron)
        ordered_fields.append(sort_tuning_curves[i])

In [ ]:
plt.plot(position.x, position.y, 'k.', ms=1)
plt.show()

In [ ]:
seq = 1
times = 'swr'
filename = info.session_id + '_swr.png'
plot_sequence(ordered_spikes,
              info.sequence['u'][times].starts[seq], 
              info.sequence['u'][times].stops[seq],
              ms_fraction=250,
              lfp=lfp,
              position=None,
              savepath=os.path.join(output_filepath, filename))

In [ ]:
seq = 1
times = 'run'
filename = info.session_id + '_run.png'
plot_sequence(ordered_spikes,
              info.sequence['u'][times].starts[seq], 
              info.sequence['u'][times].stops[seq],
              ms_fraction=150,
              lfp=None,
              position=position,
              savepath=os.path.join(output_filepath, filename))

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)

all_tuning_curves = np.zeros(neurons.tuning_shape)
for i in range(neurons.n_neurons):
    all_tuning_curves += neurons.tuning_curves[i]

pp = plt.pcolormesh(xx, yy, all_tuning_curves, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
# plt.show()
plt.savefig(os.path.join(output_filepath, info.session_id+'_tuning.png'))

In [ ]:
fig = plt.figure(figsize=(4, 3))
ax = plt.subplot(111)
start = info.sequence['u']['swr'].starts[1]
stop = info.sequence['u']['swr'].stops[1]
start_idx = nept.find_nearest_idx(lfp.time, start)
stop_idx = nept.find_nearest_idx(lfp.time, stop)
plt.plot(lfp.time[start_idx:stop_idx], lfp.data[start_idx:stop_idx], 'k', lw=0.8)
plt.axis('off')
scalebar.add_scalebar(ax, matchy=False, bbox_transform=fig.transFigure, bbox_to_anchor=(0.9, 0.2), units='ms')
plt.show()
# plt.savefig(os.path.join(output_filepath, info.session_id + '-swr1.png'), transparent=True)
# plt.close()

In [ ]:
fig = plt.figure(figsize=(4, 3))
ax = plt.subplot(111)
start = info.sequence['u']['swr'].starts[0]
stop = info.sequence['u']['swr'].stops[0]
start_idx = nept.find_nearest_idx(lfp.time, start)
stop_idx = nept.find_nearest_idx(lfp.time, stop)
plt.plot(lfp.time[start_idx:stop_idx], lfp.data[start_idx:stop_idx], 'k', lw=0.8)
plt.axis('off')
scalebar.add_scalebar(ax, matchy=False, bbox_transform=fig.transFigure, bbox_to_anchor=(0.9, 0.1), units='ms')
plt.show()
# plt.savefig(os.path.join(output_filepath, info.session_id + '-swr2.png'), transparent=True)
# plt.close()

In [ ]:
field_thresh = 1.0
fields_tunings = categorize_fields(neurons.tuning_curves, zones, xedges, yedges, 
                                   field_thresh=field_thresh)

u_line = LineString(info.u_trajectory)

u_dist = []
for neuron in fields_tunings['u']:
    yy = ycenters[np.where(fields_tunings['u'][neuron] == fields_tunings['u'][neuron].max())[0][0]]
    xx = xcenters[np.where(fields_tunings['u'][neuron] == fields_tunings['u'][neuron].max())[1][0]]

    pt = Point(xx, yy)
    if zones['u'].contains(pt):
        u_dist.append((u_line.project(pt), neuron))
        
ordered_dist_u = sorted(u_dist, key=lambda x:x[0])
sort_idx = []
for neuron in ordered_dist_u:
    sort_idx.append(neuron[1])
    
sort_spikes = []
sort_tuning_curves = []
for neuron in sort_idx:
    sort_tuning_curves.append(fields_tunings['u'][neuron])
    sort_spikes.append(decode_spikes[neuron])
    
odd_firing_idx = get_odd_firing_idx(sort_tuning_curves, max_mean_firing=2)

ordered_spikes = []
ordered_fields =[]
for i, neuron in enumerate(sort_spikes):
    if i not in odd_firing_idx:
        ordered_spikes.append(neuron)
        ordered_fields.append(sort_tuning_curves[i])

In [ ]:
counts_binsize = 0.025
time_edges = get_edges(exp_position, counts_binsize, lastbin=True)
counts = nept.get_counts(ordered_spikes, time_edges, gaussian_std=0.005)

In [ ]:
start = info.sequence['u']['run'].starts[1]
stop = info.sequence['u']['run'].stops[1]

fig = plt.figure(figsize=(10, 6))
ax = plt.subplot(111)
pp = plt.pcolormesh(counts.time_slice(start, stop).data.T, cmap='pink_r')
plt.colorbar(pp)
ax.set_xticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.get_yaxis().tick_left()
plt.show()
# plt.savefig(os.path.join(output_filepath, info.session_id + '-binned-spikes.png'), transparent=True)
# plt.close()

In [ ]:
print(info.sequence['u']['run'].starts[1], info.sequence['u']['run'].stops[1])

seq = 1
times = 'run'
filename = info.session_id + '_run.png'
plot_sequence(ordered_spikes,
              info.sequence['u'][times].starts[seq], 
              info.sequence['u'][times].stops[seq],
              ms_fraction=150,
              lfp=None,
              position=position)

In [ ]:
start_idx, stop_idx

In [ ]:
counts